# Phase 1: Baseline Model Benchmarking

This notebook systematically evaluates all baseline models on the CHARGED dataset.

**Objectives**:
- Establish performance benchmarks for all 10 baseline models
- Compare models across different cities
- Identify best baseline model per city
- Quantify auxiliary feature impact

**Models to Benchmark**:
1. Statistical: Lo, AR, ARIMA
2. Neural: LSTM, FCNN, SegRNN, FreTS, ModernTCN, MultiPatchFormer, ConvTimeNet

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

from api.dataset.common import EVDataset
from api.model.config import PredictionModel
from api.trainer.common import PredictionTrainer
from api.parsing.common import parse_args
from api.utils import calculate_regression_metrics
from experiment.utils.experiment_tracking import ExperimentTracker

print("✓ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Configuration

In [ ]:
# Experiment configuration
CONFIG = {
    'cities': ['SZH'],  # Start with Shenzhen (largest dataset)
    'models': ['moderntcn', 'convtimenet', 'lstm'],  # Priority models
    'feature': 'volume',
    'seq_lengths': [24],  # 24 hours lookback
    'pred_lengths': [1],  # 1 hour ahead prediction
    'auxiliary_modes': ['None', 'all'],  # Test with and without auxiliary features
    'folds': [1],  # Start with fold 1
    'epochs': 50,
    'batch_size': 32,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print("Experiment Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## Helper Functions

In [ ]:
def run_baseline_experiment(city, model_name, seq_len, pred_len, auxiliary, fold, epochs, batch_size, device):
    """
    Run a single baseline experiment with specified configuration.
    
    Returns:
        dict: Experiment results including metrics
    """
    print(f"\n{'='*60}")
    print(f"Running: {model_name} | {city} | seq={seq_len} | pred={pred_len} | aux={auxiliary}")
    print(f"{'='*60}")
    
    # Initialize experiment tracker
    experiment_name = f"baseline_{model_name}_{city}"
    tracker = ExperimentTracker(experiment_name, save_dir='../results/baselines')
    
    # Log hyperparameters
    tracker.log_hyperparameters({
        'city': city,
        'model': model_name,
        'feature': 'volume',
        'seq_len': seq_len,
        'pred_len': pred_len,
        'auxiliary': auxiliary,
        'fold': fold,
        'epochs': epochs,
        'batch_size': batch_size,
        'device': device
    })
    
    try:
        # Load dataset
        data_path = f'../data/{city}_remove_zero/'
        print(f"Loading dataset from {data_path}...")
        
        ev_dataset = EVDataset(
            feature='volume',
            auxiliary=auxiliary,
            data_path=data_path,
            pred_type='site',
            seq_l=seq_len,
            pre_len=pred_len
        )
        
        # Split data (80% train, 10% val, 10% test)
        ev_dataset.split_cross_validation(
            fold=fold,
            total_fold=6,
            train_ratio=0.8,
            valid_ratio=0.1
        )
        
        print(f"Dataset loaded: {ev_dataset.feat.shape[1]} sites, {ev_dataset.feat.shape[0]} timesteps")
        
        # Initialize model
        print(f"Initializing {model_name} model...")
        n_features = 1 + (ev_dataset.extra_feat.shape[1] if ev_dataset.extra_feat.size > 0 else 0)
        
        model = PredictionModel(
            model=model_name,
            seq_l=seq_len,
            pre_len=pred_len,
            n_node=ev_dataset.feat.shape[1],
            n_fea=n_features,
            device=device
        )
        
        # Initialize trainer
        trainer = PredictionTrainer(
            model=model.model,
            dataset=ev_dataset,
            epoch=epochs,
            batch_size=batch_size,
            device=device
        )
        
        # Train model (if it's a neural model)
        if model_name not in ['lo', 'ar', 'arima']:
            print(f"Training model for {epochs} epochs...")
            trainer.training()
        
        # Test and get metrics
        print("Evaluating on test set...")
        predictions, labels, metrics = trainer.test()
        
        # Log results
        tracker.log_metrics(metrics)
        tracker.save_predictions(predictions, labels)
        
        # Print results
        print(f"\n{'='*60}")
        print(f"RESULTS: {model_name} on {city}")
        print(f"{'='*60}")
        for metric_name, value in metrics.items():
            print(f"  {metric_name}: {value:.4f}")
        print(f"{'='*60}\n")
        
        # Create summary
        tracker.create_summary_table()
        
        return {
            'city': city,
            'model': model_name,
            'seq_len': seq_len,
            'pred_len': pred_len,
            'auxiliary': auxiliary,
            'fold': fold,
            **metrics,
            'status': 'success'
        }
        
    except Exception as e:
        print(f"❌ Error running experiment: {str(e)}")
        import traceback
        traceback.print_exc()
        
        return {
            'city': city,
            'model': model_name,
            'seq_len': seq_len,
            'pred_len': pred_len,
            'auxiliary': auxiliary,
            'fold': fold,
            'status': 'failed',
            'error': str(e)
        }

## Run Baseline Experiments

In [ ]:
# Run experiments for all configurations
all_results = []

total_experiments = (
    len(CONFIG['cities']) * 
    len(CONFIG['models']) * 
    len(CONFIG['seq_lengths']) * 
    len(CONFIG['pred_lengths']) * 
    len(CONFIG['auxiliary_modes']) * 
    len(CONFIG['folds'])
)

print(f"Total experiments to run: {total_experiments}\n")

experiment_count = 0
for city in CONFIG['cities']:
    for model in CONFIG['models']:
        for seq_len in CONFIG['seq_lengths']:
            for pred_len in CONFIG['pred_lengths']:
                for auxiliary in CONFIG['auxiliary_modes']:
                    for fold in CONFIG['folds']:
                        experiment_count += 1
                        print(f"\n🔬 Experiment {experiment_count}/{total_experiments}")
                        
                        result = run_baseline_experiment(
                            city=city,
                            model_name=model,
                            seq_len=seq_len,
                            pred_len=pred_len,
                            auxiliary=auxiliary,
                            fold=fold,
                            epochs=CONFIG['epochs'],
                            batch_size=CONFIG['batch_size'],
                            device=CONFIG['device']
                        )
                        
                        all_results.append(result)

print(f"\n✅ All {total_experiments} experiments completed!")

## Analyze Results

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(all_results)

# Save comprehensive results
results_df.to_csv('../results/baselines/comprehensive_baseline_results.csv', index=False)
print("✓ Saved comprehensive results to results/baselines/comprehensive_baseline_results.csv")

# Display results table
print("\n" + "="*80)
print("BASELINE BENCHMARKING RESULTS")
print("="*80)
display(results_df)

# Filter successful experiments
successful_results = results_df[results_df['status'] == 'success']

if len(successful_results) > 0:
    print(f"\nSuccessful experiments: {len(successful_results)}/{len(results_df)}")
    
    # Best model per configuration
    print("\n" + "="*80)
    print("BEST MODELS BY CONFIGURATION")
    print("="*80)
    
    for aux in CONFIG['auxiliary_modes']:
        aux_results = successful_results[successful_results['auxiliary'] == aux]
        if len(aux_results) > 0:
            best_idx = aux_results['MAE'].idxmin()
            best_model = aux_results.loc[best_idx]
            
            print(f"\nAuxiliary = {aux}:")
            print(f"  Best Model: {best_model['model']}")
            print(f"  MAE: {best_model['MAE']:.4f}")
            print(f"  RMSE: {best_model['RMSE']:.4f}")
            print(f"  R²: {best_model['R²']:.4f}")
else:
    print("\n⚠️ No successful experiments to analyze")

## Visualize Results

In [ ]:
if len(successful_results) > 0:
    # Set style
    sns.set_style('whitegrid')
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. MAE comparison
    ax1 = axes[0, 0]
    successful_results.groupby(['model', 'auxiliary'])['MAE'].mean().unstack().plot(
        kind='bar', ax=ax1, rot=45
    )
    ax1.set_title('Mean Absolute Error by Model', fontsize=12, fontweight='bold')
    ax1.set_ylabel('MAE')
    ax1.set_xlabel('Model')
    ax1.legend(title='Auxiliary Features')
    
    # 2. RMSE comparison
    ax2 = axes[0, 1]
    successful_results.groupby(['model', 'auxiliary'])['RMSE'].mean().unstack().plot(
        kind='bar', ax=ax2, rot=45
    )
    ax2.set_title('Root Mean Square Error by Model', fontsize=12, fontweight='bold')
    ax2.set_ylabel('RMSE')
    ax2.set_xlabel('Model')
    ax2.legend(title='Auxiliary Features')
    
    # 3. R² comparison
    ax3 = axes[1, 0]
    successful_results.groupby(['model', 'auxiliary'])['R²'].mean().unstack().plot(
        kind='bar', ax=ax3, rot=45
    )
    ax3.set_title('R² Score by Model', fontsize=12, fontweight='bold')
    ax3.set_ylabel('R²')
    ax3.set_xlabel('Model')
    ax3.legend(title='Auxiliary Features')
    
    # 4. Auxiliary feature impact
    ax4 = axes[1, 1]
    if len(CONFIG['auxiliary_modes']) > 1:
        aux_impact = []
        for model in CONFIG['models']:
            model_results = successful_results[successful_results['model'] == model]
            if len(model_results) >= 2:
                mae_none = model_results[model_results['auxiliary'] == 'None']['MAE'].mean()
                mae_all = model_results[model_results['auxiliary'] == 'all']['MAE'].mean()
                improvement = (mae_none - mae_all) / mae_none * 100
                aux_impact.append({'model': model, 'improvement_%': improvement})
        
        if aux_impact:
            aux_df = pd.DataFrame(aux_impact)
            aux_df.plot(x='model', y='improvement_%', kind='bar', ax=ax4, legend=False, rot=45)
            ax4.set_title('Auxiliary Features Impact (% MAE Improvement)', fontsize=12, fontweight='bold')
            ax4.set_ylabel('Improvement (%)')
            ax4.set_xlabel('Model')
            ax4.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.savefig('../results/baselines/baseline_comparison.png', dpi=300, bbox_inches='tight')
    print("\n✓ Saved visualization to results/baselines/baseline_comparison.png")
    plt.show()
else:
    print("\n⚠️ No successful experiments to visualize")

## Summary and Next Steps

In [ ]:
print("\n" + "="*80)
print("PHASE 1 COMPLETE: BASELINE BENCHMARKING")
print("="*80)

if len(successful_results) > 0:
    best_overall = successful_results.loc[successful_results['MAE'].idxmin()]
    
    print(f"\n✅ Best Overall Configuration:")
    print(f"   Model: {best_overall['model']}")
    print(f"   City: {best_overall['city']}")
    print(f"   Auxiliary: {best_overall['auxiliary']}")
    print(f"   MAE: {best_overall['MAE']:.4f}")
    print(f"   RMSE: {best_overall['RMSE']:.4f}")
    print(f"   R²: {best_overall['R²']:.4f}")
    
    print(f"\n📊 Results Summary:")
    print(f"   Total experiments: {len(results_df)}")
    print(f"   Successful: {len(successful_results)}")
    print(f"   Failed: {len(results_df) - len(successful_results)}")
    
    print(f"\n📁 Results saved to:")
    print(f"   - results/baselines/comprehensive_baseline_results.csv")
    print(f"   - results/baselines/baseline_comparison.png")
    print(f"   - Individual experiment folders in results/baselines/baseline_*/")
    
    print(f"\n🎯 Next Steps (Phase 2):")
    print(f"   1. Implement foundation model wrappers (api/model/foundation.py)")
    print(f"   2. Fine-tune MOMENT model on Shenzhen")
    print(f"   3. Target: Beat {best_overall['model']} MAE of {best_overall['MAE']:.4f}")
    print(f"   4. Goal: Achieve >10% improvement (MAE < {best_overall['MAE'] * 0.9:.4f})")
else:
    print(f"\n⚠️ All experiments failed. Please check the error messages above.")

print("\n" + "="*80)